In [1]:
import os
import json
import pandas as pd
import concurrent.futures

from openai import OpenAI
#from tqdm.notebook import tqdm
from tqdm.asyncio import tqdm

In [2]:
exec_id = 2 #[0, 1, 2, 3, 4]

In [3]:
data_path = './data/'
df = pd.read_csv(data_path + 'train.csv')

In [4]:
labels = list(set(list(df['label'])))

In [5]:
len(labels)

498968

In [6]:
left = exec_id * 100000
if exec_id <= 3:
    right = (exec_id + 1) * 100000
else:
    right = None

print(left, right)

200000 300000


In [7]:
labels = labels[left : right]
print(len(labels))

100000


In [8]:
client = OpenAI(
    #base_url = 'https://api.open-proxy.cn/v1',

    api_key = os.environ.get("OPENAI_API_KEY", ""),
    
    base_url = 'https://api.xty.app/v1',
)

In [ ]:
MODEL = 'gpt-3.5-turbo'
#MODEL = 'gpt-4-0125-preview'

out_labels = []
out_sentences = []

def process_label(label):
    messages = []
    messages.append({'role': 'system', 'content': "你是一名出色的汉语言专家，下面是一个文本生成任务，这对我的职业生涯很重要，请你认真的完成这个任务。"})
    messages.append({'role': 'system', 'content': "你只需要完成这个文本生成任务，不需要输出其他的内容。"})
    messages.append({'role': 'system', 'content': "请为这个由词语组成的句子添加必要的成分和合适的形容词、副词、语气词等形成新的意思相近的句子。"})
    messages.append({'role': 'user', 'content': label})

    response = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        temperature=0.5,
    )

    sentence = response.choices[0].message.content
    return label, sentence

# 创建一个线程池  
with concurrent.futures.ThreadPoolExecutor(max_workers = 100) as executor:  # max_workers表示并发数
    pbar = tqdm(total=len(labels))

    # 提交任务到线程池，并获取Future对象列表
    futures = {executor.submit(process_label, label): label for label in labels}

    # 遍历Future对象列表，获取结果  
    for future in concurrent.futures.as_completed(futures):
        label = futures[future]
        try:
            processed_label, sentence = future.result()
            out_labels.append(processed_label)
            out_sentences.append(sentence)
            pbar.update(1)
        except Exception as exc:
            print(f'Generated an exception: {exc}')

    pbar.close()

  1%|▌                                                                            | 757/100000 [00:16<30:26, 54.34it/s]

In [ ]:
out_labels[:5]

In [ ]:
out_sentences[:5]

In [ ]:
df_out = pd.DataFrame()
df_out['label'] = out_labels
df_out['sentence'] = out_sentences

In [ ]:
df_out.describe()

In [ ]:
df_out.to_csv(f"{data_path}data{exec_id}.csv", index = False)

In [ ]:
df_100k = pd.read_csv(data_path + 'data0.csv')

In [ ]:
df_100k.describe()

In [ ]:
df_300k = pd.read_csv(data_path + 'data1.csv')

In [ ]:
df_300k.describe()

In [ ]:
df_new = pd.concat([df_100k, df_300k], axis = 0)

In [ ]:
df_new.describe()

In [ ]:
df_new = pd.concat([df_new, df_out], axis = 0)

In [ ]:
df_new.describe()

In [ ]:
df_new.to_csv(f"{data_path}data_500k.csv", index = False)